# Vertical Chat
A sample how to build a chat for small business using:

* GPT 35
* Panel
* OpenAI


This is just a simple sample to start to understand how the OpenAI API works, and how to create Prompts. It Is really far from beign a complete solution.
We are going to introduce some interesting points:

* The roles in a conversation.
* How is the conversations’ memory preserved?

Deeper explanations in the article: [Create Your First Chatbot Using GPT 3.5, OpenAI, Python and Panel.](https://medium.com/towards-artificial-intelligence/create-your-first-chatbot-using-gpt-3-5-openai-python-and-panel-7ec180b9d7f2)

In [22]:
import os
import openai  # Ensure you import the correct module
from dotenv import load_dotenv
import panel as pn
import asyncio  # Import asyncio for async functionality

# Load environment variables from the .env file
load_dotenv()

# Get the OpenAI API key from the environment
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Set the API key for OpenAI
openai.api_key = OPENAI_API_KEY

pn.extension()

# Define the initial context with the system prompt for the OrderBot role
context = [{'role': 'system', 'content': """
Act as an OrderBot, you work collecting orders in a delivery-only fast food restaurant called
My Dear Frankfurt. First welcome the customer, in a very friendly way, then collect the order. 
You wait to collect the entire order, beverages included, then summarize it and check for a final 
time if everything is ok or the customer wants to add anything else. Finally, you collect the payment.
Make sure to clarify all options, extras, and sizes to uniquely identify the item from the menu.
You respond in a short, very friendly style. The menu includes:

- Burger: 12.95, 10.00, 7.00
- Frankfurt: 10.95, 9.25, 6.50
- Sandwich: 11.95, 9.75, 6.75
- Fries: 4.50, 3.50
- Salad: 7.25

Toppings:
- Extra cheese: 2.00
- Mushrooms: 1.50
- Martra sausage: 3.00
- Canadian bacon: 3.50
- Romesco sauce: 1.50
- Peppers: 1.00

Drinks:
- Coke: 3.00, 2.00, 1.00
- Sprite: 3.00, 2.00, 1.00
- Vichy Catalan: 5.00
"""}]

async def callback(contents: str, user: str, instance: pn.chat.ChatInterface):
    # Add the user's message to the context
    context.append({'role': 'user', 'content': contents})
    
    # Generate a response using the existing context
    try:
        response = await openai.ChatCompletion.acreate(
            model="gpt-3.5-turbo",
            messages=context,
            stream=True,
        )

        message = ""
        async for chunk in response:
            # Check if 'choices' and 'delta' are present in the response
            if 'choices' in chunk and chunk.choices:
                part = chunk.choices[0].delta.get('content', '')
                if part:
                    message += part
                    yield message
            else:
                yield "Sorry, I couldn't generate a response."
    
    except Exception as e:
        yield f"An error occurred: {str(e)}"

def main():
    chat_interface = pn.chat.ChatInterface(
        callback=callback,
        callback_user="OrderBot",
        help_text="Welcome to My Dear Frankfurt! How can I help you with your order today?",
    )
    
    template = pn.template.FastListTemplate(
        title="OpenAI OrderBot",
        header_background="#212121",
        main=[chat_interface]
    )

    template.servable()
    
    # Serve the Panel app
    pn.serve(template, show=True)

if __name__ == "__main__":
    main()  # Directly call main() without asyncio


Launching server at http://localhost:52769


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [24]:
import os
import openai  # Ensure you import the correct module
from dotenv import load_dotenv
import panel as pn
import asyncio  # Import asyncio for async functionality

# Load environment variables from the .env file
load_dotenv()

# Get the OpenAI API key from the environment
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Set the API key for OpenAI
openai.api_key = OPENAI_API_KEY

pn.extension()

# Define the initial context with the first prompt variation
context = [{'role': 'system', 'content': """
You are the OrderBot at a charming fast food restaurant called My Dear Frankfurt. 
Greet every customer with a warm smile and a friendly hello. Ask them what delicious items they would like to order today, 
encouraging them to share their preferences for drinks and toppings. 
Once they have finished their order, summarize it clearly, double-checking if they want to add anything else before finalizing the payment.
"""}]

async def callback(contents: str, user: str, instance: pn.chat.ChatInterface):
    # Add the user's message to the context
    context.append({'role': 'user', 'content': contents})

    # Generate a response using the existing context
    try:
        response = await openai.ChatCompletion.acreate(
            model="gpt-3.5-turbo",
            messages=context,
            stream=True,
        )

        message = ""
        async for chunk in response:
            # Check if 'choices' and 'delta' are present in the response
            if 'choices' in chunk and chunk.choices:
                part = chunk.choices[0].delta.get('content', '')
                if part:
                    message += part
                    yield message
            else:
                yield "Sorry, I couldn't generate a response."

    except Exception as e:
        yield f"An error occurred: {str(e)}"

def main():
    chat_interface = pn.chat.ChatInterface(
        callback=callback,
        callback_user="OrderBot",
        help_text="Welcome to My Dear Frankfurt! How can I help you with your order today?",
    )
    
    template = pn.template.FastListTemplate(
        title="OpenAI OrderBot",
        header_background="#212121",
        main=[chat_interface]
    )

    template.servable()
    
    # Serve the Panel app
    pn.serve(template, show=True)

if __name__ == "__main__":
    main()  # Directly call main() without asyncio



Launching server at http://localhost:52821


In [25]:
 #Interactive and Engaging
import os
import openai  # Ensure you import the correct module
from dotenv import load_dotenv
import panel as pn
import asyncio  # Import asyncio for async functionality

# Load environment variables from the .env file
load_dotenv()

# Get the OpenAI API key from the environment
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Set the API key for OpenAI
openai.api_key = OPENAI_API_KEY

pn.extension()

# Define the initial context with the second prompt variation
context = [{'role': 'system', 'content': """
Welcome to My Dear Frankfurt! As the OrderBot, your role is to create an engaging experience for every customer. 
Start by saying hello and asking how they are doing today. Prompt them to order by suggesting some popular items, 
and invite them to customize their meal with various toppings and drink options.
"""}]

async def callback(contents: str, user: str, instance: pn.chat.ChatInterface):
    # Add the user's message to the context
    context.append({'role': 'user', 'content': contents})

    # Generate a response using the existing context
    try:
        response = await openai.ChatCompletion.acreate(
            model="gpt-3.5-turbo",
            messages=context,
            stream=True,
        )

        message = ""
        async for chunk in response:
            # Check if 'choices' and 'delta' are present in the response
            if 'choices' in chunk and chunk.choices:
                part = chunk.choices[0].delta.get('content', '')
                if part:
                    message += part
                    yield message
            else:
                yield "Sorry, I couldn't generate a response."

    except Exception as e:
        yield f"An error occurred: {str(e)}"

def main():
    chat_interface = pn.chat.ChatInterface(
        callback=callback,
        callback_user="OrderBot",
        help_text="Welcome to My Dear Frankfurt! How can I help you with your order today?",
    )
    
    template = pn.template.FastListTemplate(
        title="OpenAI OrderBot",
        header_background="#212121",
        main=[chat_interface]
    )

    template.servable()
    
    # Serve the Panel app
    pn.serve(template, show=True)

if __name__ == "__main__":
    main()  # Directly call main() without asyncio


Launching server at http://localhost:52838


In [26]:
#Fun and Casual

import os
import openai  # Ensure you import the correct module
from dotenv import load_dotenv
import panel as pn
import asyncio  # Import asyncio for async functionality

# Load environment variables from the .env file
load_dotenv()

# Get the OpenAI API key from the environment
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Set the API key for OpenAI
openai.api_key = OPENAI_API_KEY

pn.extension()

# Define the initial context with the third prompt variation
context = [{'role': 'system', 'content': """
Hey there! You’re the OrderBot for a trendy fast food spot called My Dear Frankfurt. 
Kick things off with a cheerful greeting and ask the customer what they’re craving today! 
Make it fun by suggesting combos or popular items.
"""}]

async def callback(contents: str, user: str, instance: pn.chat.ChatInterface):
    # Add the user's message to the context
    context.append({'role': 'user', 'content': contents})

    # Generate a response using the existing context
    try:
        response = await openai.ChatCompletion.acreate(
            model="gpt-3.5-turbo",
            messages=context,
            stream=True,
        )

        message = ""
        async for chunk in response:
            # Check if 'choices' and 'delta' are present in the response
            if 'choices' in chunk and chunk.choices:
                part = chunk.choices[0].delta.get('content', '')
                if part:
                    message += part
                    yield message
            else:
                yield "Sorry, I couldn't generate a response."

    except Exception as e:
        yield f"An error occurred: {str(e)}"

def main():
    chat_interface = pn.chat.ChatInterface(
        callback=callback,
        callback_user="OrderBot",
        help_text="Welcome to My Dear Frankfurt! How can I help you with your order today?",
    )
    
    template = pn.template.FastListTemplate(
        title="OpenAI OrderBot",
        header_background="#212121",
        main=[chat_interface]
    )

    template.servable()
    
    # Serve the Panel app
    pn.serve(template, show=True)

if __name__ == "__main__":
    main()  # Directly call main() without asyncio


Launching server at http://localhost:52852


Summary Report of Findings
Title: Analysis of OrderBot Prompts
Objective:
The aim of this exercise was to explore different prompt styles for an OrderBot at My Dear Frankfurt and analyze their effectiveness in facilitating customer interactions.

Prompt Variations Tested:

Warm and Welcoming: Focused on friendliness and clarity.
Interactive and Engaging: Encouraged engagement and customization.
Fun and Casual: Aimed for a relaxed and enjoyable interaction.

Findings:

Effective Variations:

The Warm and Welcoming and Fun and Casual prompts elicited highly positive responses. Customers appreciated the friendly tone, which made the interaction feel personalized and engaging.
The Interactive and Engaging prompt was also effective, as it prompted the OrderBot to offer suggestions, which encouraged customer input and interaction.
Variations That Didn't Work Well:

While all prompts performed relatively well, some responses showed slight inconsistencies. For instance, in the Interactive and Engaging variation, the model occasionally misinterpreted customer requests for special dietary preferences or asked for clarification on menu items not present in the original list.
In a few cases, the chatbot generated fictional toppings or drinks, indicating a tendency for hallucination when given open-ended questions about meal combinations.
Lessons Learned:

Importance of Tone: The tone of the prompts significantly influenced the customer experience. Casual and friendly prompts led to better engagement compared to a strictly professional tone.
Specificity in Prompts: Providing detailed instructions in prompts yielded better results. However, being too vague led to unexpected outcomes, such as hallucinations.
Iterative Testing: Experimenting with different styles demonstrated that iterative testing can refine prompts for better accuracy and user satisfaction. Future prompts can be fine-tuned based on the insights gathered from these variations.
Conclusion
The exercise highlighted the significance of prompt design in shaping user interactions with chatbots. Tailoring the tone and specificity of prompts can lead to improved customer experiences, while continuous testing and learning from generated responses can further optimize the chatbot's capabilities.

